In [1]:
import os, random
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
os.path.expanduser = lambda path: './'

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!pip install tensorflow

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/228.5 MB 197.7 kB/s eta 0:16:30

In [2]:
batch_size = 128
num_classes = 10
epochs = 60

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


# Define Model 

In [3]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
#tf.set_random_seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)

kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 673,802
Trainable params: 671,754
Non-trainable params: 2,048
_________________________________________________________________


# Start Training

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 16s 11ms/step - loss: 0.7511 - accuracy: 0.7441 - val_loss: 0.4801 - val_accuracy: 0.8393
Epoch 2/60
375/375 [==============================] - 4s 9ms/step - loss: 0.3993 - accuracy: 0.8573 - val_loss: 0.3964 - val_accuracy: 0.8575
Epoch 3/60
375/375 [==============================] - 4s 12ms/step - loss: 0.3441 - accuracy: 0.8788 - val_loss: 0.3657 - val_accuracy: 0.8681
Epoch 4/60
375/375 [==============================] - 4s 10ms/step - loss: 0.3182 - accuracy: 0.8871 - val_loss: 0.3520 - val_accuracy: 0.8717
Epoch 5/60
375/375 [==============================] - 4s 10ms/step - loss: 0.2891 - accuracy: 0.8959 - val_loss: 0.3532 - val_accuracy: 0.8713
Epoch 6/60
375/375 [==============================] - 4s 9ms/step - loss: 0.2702 - accuracy: 0.9059 - val_loss: 0.3443 - val_accuracy: 0.8752
Epoch 7/60
375/375 [==============================] - 4s 10ms/step - loss: 0.2591 - accuracy: 0.9086 - val_loss: 0.3345 - val_accuracy: 0.8770


Epoch 58/60
375/375 [==============================] - 4s 11ms/step - loss: 0.0262 - accuracy: 0.9955 - val_loss: 0.4368 - val_accuracy: 0.8851
Epoch 59/60
375/375 [==============================] - 4s 11ms/step - loss: 0.0247 - accuracy: 0.9953 - val_loss: 0.4243 - val_accuracy: 0.8858
Epoch 60/60
375/375 [==============================] - 4s 11ms/step - loss: 0.0256 - accuracy: 0.9947 - val_loss: 0.4256 - val_accuracy: 0.8857


# Calculate accuracy 

In [5]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 1s 2ms/step - loss: 0.4524 - accuracy: 0.8825
0.8824999928474426
Accuracy: 88.25%

